## Problem Set 3

Total points: 20

Fraction of problem set two points: 25\%

**Concepts**: here, you're going to start using the SIP H-2A employer jobs and debarment data to practice exact matching,  regex, and fuzzy matching

In [ ]:
## helpful packages
import pandas as pd
import numpy as np
import random
import re
import recordlinkage


## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


# 1. Regex and exact matching (8 points)



# 1.1 Load data on debarments and job postings (0 points)

Load the following datasets stored in `pset2_part1_inputdata`
    
- Historical H2A debarments (debar.csv); call this `debar`
- Q1 2021 H2A job postings (jobs.csv); call this `jobs`


### 1.2 Try exact merge on business name  (2 points)

- Use the `EMPLOYER_NAME` field of jobs
- Use the `Name` field of debar

A. Use pd.merge with an inner join on those fields to see whether there are any exact matches. 

B. If there are matches, print the name in each dataset, date range of debarment (`Start date` and `End date` in `debar`) and location from each data (`City, State` in `debar` and `EMPLOYER_CITY` and `EMPLOYER_STATE` in `jobs`)

**Resources**:
    - Slides 17-19 here: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/slides/qss20_s21_class4.pdf 
    - Code here: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/03_merging_session1.ipynb 


## 1.3 Targeted regex

You want to see if you can increase the exact match rate with some basic cleaning of each 
of the name fields.


### 1.3.1 Converting to upper (2 points)

A. Convert the names `EMPLOYER_NAME` and `Name` to uppercase using list comprehension rather than df.varname.str.upper()

B. Print a random sample of 15 values of each result

C. Assign them back to the original data, writing over the original columns

**Resources**:
    - `new_colnames` line of code here for example of list comprehension: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/04_basicregex_formerging.ipynb
    - Sampling from a list without replacement using the `random` module: https://note.nkmk.me/en/python-random-choice-sample-choices/ 
    

### 1.3.2 Cleaning up punctuation (4 points)

You notice that INC, CO, and LLC are sometimes followed by a . but sometimes not

A. For each dataset, write a regex pattern to remove the . but only if it's preceded by INC, LLC, or CO 

Make sure LLC, INC, CO remain part of the string but just without the dot

B. Test the pattern on the positive and negative example and print the result. The first positive example should return `CISCO PRODUCE INC`; the second positive example should return `AVOYELLES HONEY CO, LLC` the negative example should return `E.V. RANCH LLP` (so leave the dots between E.V. in)

C. Execute on (1) Name in `debar` (can just write over the col) and (3) EMPLOYER_NAME  in `jobs`, making sure to use the uppercase versions of the variables

**Hint**: https://stackoverflow.com/questions/7191209/python-re-sub-replace-with-matched-content

**Resources**:
    - Regex slides are 22-40: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/slides/qss20_s21_class4.pdf 
    - Regex code: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/04_basicregex_formerging.ipynb

In [ ]:
pos_example_1 = "CISCO PRODUCE INC."
pos_example_2 = "AVOYELLES HONEY CO., LLC"
neg_example = "E.V. RANCH LLP"

### 1.3.3 OPTIONAL extra credit - regex to separate companies from individuals (2 points)

You notice some employers in `debar` have both the name of the company and the name of individual, e.g.:
    
COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMSON (INDIVIDUAL)*

Use the uppercase/cleaned version of the Name in `debar`

A. Write a regex pattern that does the following:
    - Captures the pattern that occurs before COMPANY if (COMPANY) is in string-- so in above, extracts COUNTY FAIR FARM 
    - Captures the pattern that occurs before INDIVIDUAL if (INDIVIDUAL) is also in string -- so in above, extracts ANDREW WILLIAMSON (so omit the and)
    
B. Test the pattern on `pos_example` and `neg_example`-- make sure former returns a list (if using find.all) or match object (if using re.search) with the company name and individual name separated out; make sure latter returns empty
    
C. Iterate over the cleaned `Name` column in debar and execute the regex. Use it to create two new columns in debar:
    - A column for company (full Name string if left as is; pattern before COMPANY if one extracted)
    - A column for individual (full Name string if left as is; pattern before INDIVIDUAL if one extracted)
    
D. Print those columns for the rows containing the negative example and positive example

**Hint**: for step A, you can either use re.search, re.match, or re.findall; don't worry about matching B&R Harvesting and Paul Cruz (Individual)

For step C, you probably want to use a loop or function that uses if else to do different things if a match is found or not


In [ ]:
pos_example = "COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMSON (INDIVIDUAL)*"
neg_example = "CISCO PRODUCE INC"

# 1.4 Fuzzy matching to match debarments to jobs (12 points)


### 1.4.1 Preprocessing location (2 points)

You want to block on state but notice that states in `debar` have a mix of two digit codes and full state names (e.g., GA versus Georgia) while states in `jobs` are all two-digit codes

A. Run the code below to load the `states` crosswalk. Use that crosswalk to create a field in `debar` that has the two-digit state abbreviation for all locations (hint: you may need to first split the string on the ", " or use str.replace to extract the state)

B. Use an `assert` statement to check that all the states in `debar` are two-digits after the cleaning

**Notes**: you can filter out states that are NaN

In [ ]:
## code to load state crosswalk
## source- https://towardsdatascience.com/state-name-to-state-abbreviation-crosswalks-6936250976c
cw_location = 'http://app02.clerk.org/menu/ccis/Help/CCIS%20Codes/'
cw_filename = 'state_codes.html'
states = pd.read_html(cw_location + cw_filename)[0]
states.head()

### 1.4.2 step by step fuzzy matching (4 points)



A. Write fuzzy matching code (don't yet put inside a user-defined function) that:

- Blocks on two-digit state code
- Finds matches based on similarity between the employer name (`Name`) in `debar` (uppercase and cleaned) and `EMPLOYER_NAME` in `jobs` (uppercase and cleaned). You can choose which distance metric and threshold to use (feel free to set a threshold low enough to get some matches even if that leads to some false positives).

For the steps after compute, just take any match with non-zero value rather than using a classifier (so skip the k-means or e-m step in the example code)

B. Print the matches and comment on examples of ones that seem like true positives and ones that seem like false matches

**Resources**:

- Solutions code here: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/05_merging_session2_activitysolutions.ipynb 
- Example code here: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/05_merging_session2_codeexample.ipynb


**Hint**: you may need to deduplicate multiple records in the datasets for the recordlinkage package to work. See drop_duplicates within pandas and subset command

### 1.4.3 write a fuzzy matching function (6 points)

You want to see how the matches change if you add the town and not only state as a field and also want to automate the process of matching a bit to try different distance thresholds.

A. Extract the City from the `City, State` column of debar

B. Convert that new `city` column to uppercase and convert the `EMPLOYER_CITY` column in jobs to uppercase

C. Write a function surrounding the code in `recordlinkage` (so you don't need to recode the package from scratch) that (1) takes in each dataset, (2) blocks on two-digit state, and (3) fuzzy matches on employer name and employer city

D. Execute the function with a couple different string distance thresholds and print the results of each

5 out of 6 points: function takes arguments for input datasets, varname to block on, two varnames to fuzzy match on, string distance function, and string distance threshold
    
6 out of 6: above but function is also general enough that it takes a variable # of strings to match on--- so should work if you either execute just using employer name or also work if you execute using employer name and employer city